# Notebook Objective

This notebook is for running the DNAseq pipeline to transform .fastq files into .maf files. We use %%bash to indicate that the code we run is actually bash commands on the Arashi Linux server. This helps significantly with *reproducibility*.

Run each cell individually; some pieces of code will take a long time to run. **Ensure you are using the DNAseq-kernel**, which corresponds to the DNAseq environment. This should have all the packages necessary to run the pipeline, including Oliver, the manager for Cromwell (the server you submit each pipeline job to).

Link to the official pipeline documentation: https://github.com/isambmi/running-with-oliver-cromwell/tree/main/DNAseq-short-variant-discovery

Note: you should have already created a .csv file containing various information about each .fastq file, including paths. Because we're working with colorectal cancer, there will be **MANY .fastq for a single sample**. Don't let this confuse you and document the .csv properly to avoid downstream error. A Python script will be run on the server using this notebook to generate a proper input json file for the first step of the pipeline.

## Table of Contents <a name='tob' />

Go to <a href=#SCMA>SCMA</a>
<br>
Go to <a href=#pre-processing>Pre-Processing</a>
<br>
Go to <a href=#pon>PON</a>
<br>
Go to <a href=#mutect>Mutect</a>
<br>
Go to <a href=#agg-maf>Aggregate mafs</a>

# Declare Path Variables

First, we'll instantiate all the necessary path variables that we'll use for this pipeline. Find these paths by navigating the Linux server and copying-pasting them here. This should be the **only cell you have to edit each time you run the pipeline**.

These variables will mostly contain the paths to your chosen inputs/input-directories and output-directories. They will also contain group names (for group submissions to Cromwell).

### Edit these variables

In [1]:
raw_file_dir = "dcr_1"
group_name = "DCR"
python_script_name = "gen_gatk_inputs-4.py"

Write the group names in progress, but might not currently being used in the run:

    DCR-0003-0011-0027_2

    DCR-002-0045-0056_2
    
    DCR-090-102-105_2
    
    DCR-103_2
    
    2 is for the second run.
    

## Clear Previous Runs

Run this code block to clear previous runs of Cromwell that you no longer need. This is to ensure there is space for more runs of the pipeline down the line. For instance, you can run this every time you complete a pipeline. It's also good to run this to clear out failed runs.

In [4]:
%%bash -s "$group_name"

source activate
conda activate DNAseq

# move to the directory with the script to clear previous runs
cd /home/cromwell-scripts/

# ./oliver-rm-bygroup.sh -g $1_scma_normal
# ./oliver-rm-bygroup.sh -g $1_scma_tumor
# ./oliver-rm-bygroup.sh -g $1_pp_normal
# ./oliver-rm-bygroup.sh -g $1_pp_tumor
# ./oliver-rm-bygroup.sh -g $1_pon
# ./oliver-rm-bygroup.sh -g $1_mutect
./oliver-rm-bygroup.sh -g DCR-090-102-105_2_mutect_2

Could not find conda environment: $group_name
You can list all discoverable environments with `conda info --envs`.

rm: cannot remove '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/execution/interval-files/0000-scattered.interval_list': Permission denied
rm: cannot remove '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/execution/interval-files/0001-scattered.interval_list': Permission denied
rm: cannot remove '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/execution/interval-files/0002-scattered.interval_list': Permission denied
rm: cannot remove '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/execution/interval-files/0003-scattered.interval_list': Permission denied
rm: cannot remove '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/execution/interval-files/0004-scattere

removed '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/execution/0000-scattered.interval_list'
removed '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/execution/0001-scattered.interval_list'
removed '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/execution/0002-scattered.interval_list'
removed '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/execution/0003-scattered.interval_list'
removed '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/execution/0004-scattered.interval_list'
removed '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/execution/0005-scattered.interval_list'
removed '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/execution/0006-scattered.interval_list'
removed '/home/cromw

rm: cannot remove '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/execution/glob-0fc990c5ca95eebc97c4c204e3e303e1/cromwell_glob_control_file': Permission denied
rm: cannot remove '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/execution/glob-0fc990c5ca95eebc97c4c204e3e303e1/0000-scattered.interval_list': Permission denied
rm: cannot remove '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/execution/glob-0fc990c5ca95eebc97c4c204e3e303e1/0001-scattered.interval_list': Permission denied
rm: cannot remove '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/execution/glob-0fc990c5ca95eebc97c4c204e3e303e1/0002-scattered.interval_list': Permission denied
rm: cannot remove '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/execution/glob-0fc990c5ca95eebc97c4c204e3e303e1/0003-scattered.i

rm: cannot remove '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/execution/glob-0fc990c5ca95eebc97c4c204e3e303e1/0039-scattered.interval_list': Permission denied
rm: cannot remove '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/execution/glob-0fc990c5ca95eebc97c4c204e3e303e1/0040-scattered.interval_list': Permission denied


removed '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/execution/glob-0fc990c5ca95eebc97c4c204e3e303e1.list'
removed '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/execution/rc'
removed '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/inputs/1500889447/human_g1k_v37_decoy.fasta.fai'
removed '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/inputs/1500889447/hg19-wgs_calling_regions.v1.interval_list'
removed '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/inputs/1500889447/human_g1k_v37_decoy.fasta'
removed '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-SplitIntervals/inputs/1500889447/human_g1k_v37_decoy.dict'
removed '/home/cromwell-executions/Mutect2/da367131-0f38-4a1f-9569-150e37ec3aa1/call-M2/shard-36/execution/bamout.bam'
removed '/home

...

removed '/home/cromwell-executions/Mutect2/5d409eb6-2198-4a0f-8ef8-5701740ae2bc/call-M2/shard-26/inputs/1500889447/af-only-gnomad.raw.sites.vcf'


rm: cannot remove '/home/cromwell-executions/Mutect2/5d409eb6-2198-4a0f-8ef8-5701740ae2bc/call-Funcotate/execution/datasources_dir/cosmic/metadata.txt': Permission denied
rm: cannot remove '/home/cromwell-executions/Mutect2/5d409eb6-2198-4a0f-8ef8-5701740ae2bc/call-Funcotate/execution/datasources_dir/gnomAD_exome.tar.gz': Permission denied
rm: cannot remove '/home/cromwell-executions/Mutect2/5d409eb6-2198-4a0f-8ef8-5701740ae2bc/call-Funcotate/execution/datasources_dir/gnomAD_genome.tar.gz': Permission denied
rm: cannot remove '/home/cromwell-executions/Mutect2/5d409eb6-2198-4a0f-8ef8-5701740ae2bc/call-Funcotate/execution/datasources_dir/gencode_xhgnc/hg38/gencode_xhgnc_v90_38.hg38.tsv': Permission denied
rm: cannot remove '/home/cromwell-executions/Mutect2/5d409eb6-2198-4a0f-8ef8-5701740ae2bc/call-Funcotate/execution/datasources_dir/gencode_xhgnc/hg38/gencode_xhgnc.config': Permission denied
rm: cannot remove '/home/cromwell-executions/Mutect2/5d409eb6-2198-4a0f-8ef8-5701740ae2bc/call-

removed '/home/cromwell-executions/Mutect2/5d409eb6-2198-4a0f-8ef8-5701740ae2bc/call-Funcotate/execution/DCR105_TA.b37-filtered.annotated.maf.idx'
removed '/home/cromwell-executions/Mutect2/5d409eb6-2198-4a0f-8ef8-5701740ae2bc/call-Funcotate/execution/rc'
removed '/home/cromwell-executions/Mutect2/5d409eb6-2198-4a0f-8ef8-5701740ae2bc/call-Funcotate/inputs/1500889447/human_g1k_v37_decoy.fasta.fai'
removed '/home/cromwell-executions/Mutect2/5d409eb6-2198-4a0f-8ef8-5701740ae2bc/call-Funcotate/inputs/1500889447/human_g1k_v37_decoy.dict'
removed '/home/cromwell-executions/Mutect2/5d409eb6-2198-4a0f-8ef8-5701740ae2bc/call-Funcotate/inputs/1639117019/DCR105_TA.b37-filtered.vcf.idx'


rm: cannot remove '/home/cromwell-executions/Mutect2/5d409eb6-2198-4a0f-8ef8-5701740ae2bc/call-M2Complete/execution/rc': Permission denied


removed '/home/cromwell-executions/Mutect2/5d409eb6-2198-4a0f-8ef8-5701740ae2bc/call-M2/shard-13/inputs/1500889447/human_g1k_v37_decoy.fasta'
removed '/home/cromwell-executions/Mutect2/5d409eb6-2198-4a0f-8ef8-5701740ae2bc/call-M2/shard-28/inputs/466194140/DCR105_NA.b37.bam'
removed '/home/cromwell-executions/Mutect2/5d409eb6-2198-4a0f-8ef8-5701740ae2bc/call-M2/shard-24/inputs/-3882270/DCR105_TA.b37.bam'
removed '/home/cromwell-executions/Mutect2/5d409eb6-2198-4a0f-8ef8-5701740ae2bc/call-M2/shard-3/inputs/466194140/DCR105_NA.b37.bam'
removed '/home/cromwell-executions/Mutect2/5d409eb6-2198-4a0f-8ef8-5701740ae2bc/call-M2/shard-33/inputs/1500889447/af-only-gnomad.raw.sites.vcf'
removed '/home/cromwell-executions/Mutect2/5d409eb6-2198-4a0f-8ef8-5701740ae2bc/call-M2/shard-11/inputs/1500889447/human_g1k_v37_decoy.fasta'
removed '/home/cromwell-executions/Mutect2/5d409eb6-2198-4a0f-8ef8-5701740ae2bc/call-M2/shard-32/inputs/1500889447/human_g1k_v37_decoy.fasta'
removed '/home/cromwell-executio

### Do not have to edit these variables

In [2]:
input_csv = f"/home/ckane/DAS/{raw_file_dir}/csv_input/{group_name}.csv"

# the inputs for scma step. Separated by normal vs tumor
scma_normal_input_dir = f"/home/ckane/DAS/{raw_file_dir}/cromwell_input/{group_name}/scma/normal" 
scma_tumor_input_dir = f"/home/ckane/DAS/{raw_file_dir}/cromwell_input/{group_name}/scma/tumor" 

# receives output fofn lists of scma step, ubams, split by normal vs tumor
scma_normal_output_dir = f"/home/ckane/DAS/{raw_file_dir}/ubam/normal/{group_name}"
scma_tumor_output_dir = f"/home/ckane/DAS/{raw_file_dir}/ubam/tumor/{group_name}"

# input for preprocessing is the output from scma step, still separated by normal vs tumor
pp_normal_input_dir = scma_normal_output_dir
pp_tumor_input_dir = scma_tumor_output_dir

# receives bam output from preprocessing step
pp_normal_output_dir = f"/home/ckane/DAS/{raw_file_dir}/bam/b37-n/{group_name}"
pp_tumor_output_dir = f"/home/ckane/DAS/{raw_file_dir}/bam/b37-t/{group_name}"

# the input file for pon and the output directory for pon
# should be a single pon file named pon.json, containing a list of all normals in the run
# the file already contains the paths specified by pp_normal_output_dir
pon_input_file = f"/home/ckane/DAS/{raw_file_dir}/cromwell_input/{group_name}/pon/pon.json"
pon_output_dir = f"/home/ckane/DAS/{raw_file_dir}/pon/{group_name}"

# the input and output directories for m2
# input directory contains one input .json per sample
m2_input_dir = f"/home/ckane/DAS/{raw_file_dir}/cromwell_input/{group_name}/m2"
m2_output_dir = f"/home/ckane/DAS/{raw_file_dir}/maf/{group_name}"

print(input_csv)
print(scma_normal_input_dir, scma_tumor_input_dir, scma_normal_output_dir, scma_tumor_output_dir)
print(pp_normal_input_dir, pp_tumor_input_dir, pp_normal_output_dir, pp_tumor_output_dir)
print(pon_input_file, pon_output_dir)
print(m2_input_dir, m2_output_dir)

/home/ckane/DAS/dcr_1/csv_input/DCR.csv
/home/ckane/DAS/dcr_1/cromwell_input/DCR/scma/normal /home/ckane/DAS/dcr_1/cromwell_input/DCR/scma/tumor /home/ckane/DAS/dcr_1/ubam/normal/DCR /home/ckane/DAS/dcr_1/ubam/tumor/DCR
/home/ckane/DAS/dcr_1/ubam/normal/DCR /home/ckane/DAS/dcr_1/ubam/tumor/DCR /home/ckane/DAS/dcr_1/bam/b37-n/DCR /home/ckane/DAS/dcr_1/bam/b37-t/DCR
/home/ckane/DAS/dcr_1/cromwell_input/DCR/pon/pon.json /home/ckane/DAS/dcr_1/pon/DCR
/home/ckane/DAS/dcr_1/cromwell_input/DCR/m2 /home/ckane/DAS/dcr_1/maf/DCR


# Create a new directory to work in (only run this once)

We have a template directory that we'll simply copy into the DAS directory. We'll use the chosen name above. You only have to do this ONCE for each set of samples. You can do multiple runs (group submits) in the same directory!

In [7]:
%%bash -s "$raw_file_dir"

# we just passed in our variable test_name to our bash cell, allowing us to use it in our commands
# note that when we call on it, we use its "index"
# the indices start at 1 and increase depending on how many variables you need to pass

cp -r /home/ckane/crc_template/ /home/ckane/DAS/$1

In [12]:
%%bash -s "$scma_normal_output_dir" "$scma_tumor_output_dir" "$pp_normal_output_dir" "$pp_tumor_output_dir" "$pon_output_dir" "$m2_output_dir"

# generate the group-specific output directories for this run of the pipeline
mkdir $1 $2 $3 $4 $5 $6

## Generate config.ini files for normal and tumor

In [20]:
# Define the configuration text as a multi-line string
config_text = f"""
[RUNTIME]
; these usually change with each batch of data
groupname = {group_name}
; this is usually a reference to the accession or source of the data
input_file = {input_csv}
ref_version = b37
; option of b37 or hg38
raw_file_dir = /home/ckane/DAS/{raw_file_dir}
; parent directory where the BAM, PON, and MAF files are expected to be
;; note: the pipeline does not move these files here automatically, the files must be moved manually
send_email_for_all = False
; sends email for each workflow that completes, this can get overwhelming if processing a large batch of data. By default, an email with always be sent when the last workflow inputted has completed
;; value is either True or False
interval_file = /home/imibrahim/refs/b37/hg19-wgs_calling_regions.v1.interval_list
;; path to the interval file. Leave empty to use the default interval files for b37 and hg38 (WGS)
;; this specified path is for WGS! Check if the pon and mutect jsons have this as the path.

[USERCONFIG]
; user specific settings: you would usually only change this the first time you get this conf
email = cadekane@hawaii.edu
base_output_dir = /home/ckane/DAS/{raw_file_dir}/cromwell_input
; location where JSONs generated by this script will be output
;; the script will save all JSONs in a subdirectory of the same groupname e.g. /base/output/dir/groupname

[RAWFILEDIRS]
; directory structure where the raw files are expected to be on the server: this would also usually only be set once, unless you change directory structures for each run
n_dir = bam/b37-n/{group_name}
; e.g. /raw/file/dir/bam/b37-n
t_dir = bam/b37-t/{group_name}
pon_dir = pon/{group_name}
; e.g. /raw/file/dir/pon/

[SERVERCONFIG]
; server specific run: these would rarely change
gatk_docker = arashi-gatk-426
; this refers to the container (singularity/docker) used for the pipeline. These containers are located at /home/imibrahim/SIFs
ref_dir = /home/imibrahim/refs
; this is the location of the reference files on ARASHI. Don't change unless intending to use the pipeline with own set of references not defined here
"""

# Specify the file path
file_path = f"{group_name}_config.ini"

# Write the configuration text to the file
with open(file_path, 'w') as file:
    file.write(config_text)

In [22]:
%%bash -s "$raw_file_dir" "$group_name"

# move the config.ini file from the jupyter notebook directory to the working directory
mv /home/ckane/DAS/jupyter-test/$2_config.ini /home/ckane/DAS/$1

# Generate Input JSONS using the Python script and config.ini files

When you run this Python script, it will CREATE new directories on the Linux server for your input files to go into (described in the script). You should manually copy or move the script into your raw_file_dir.

Note: this requires pandas in your conda environment.

In [23]:
%%bash -s "$raw_file_dir" "$python_script_name" "$group_name"

# change to the directory containing your Python script and config.ini file
cd /home/ckane/DAS/$1

# run the python script with config.ini file as its argument
python $2 $3_config.ini

Reading keys in config.ini ...
Generated JSONs will be saved to: /home/ckane/DAS/dcr_1/cromwell_input/DCR-103_2


Loading input file: /home/ckane/DAS/dcr_1/csv_input/DCR-103_2.csv


=== INPUT GEN START ===




Generating input for SCMA ...
64 input files for SCMA saved to: /home/ckane/DAS/dcr_1/cromwell_input/DCR-103_2/scma


Generating input for Panel of Normals ...
PoN input saved to:
/home/ckane/DAS/dcr_1/cromwell_input/DCR-103_2/pon/pon.json


Generating input for m2 ...
1 input JSON saved to:
/home/ckane/DAS/dcr_1/cromwell_input/DCR-103_2/m2


=== INPUT GEN END ===


## Organize Samples Into Normal and Tumor Directories

It's important that we submit our samples separately by normal and tumor, to simplify things downstream. In this current example, we are working with three CRC samples, each with many lane-read fastqs. This does not change things and we can process several samples at the same time. We will, however, split them up by normal and tumor classification.

In [24]:
%%bash -s "$raw_file_dir" "$group_name"

# we split the scma input .jsons into normal and tumor directories
# it is only necessary for us to do this for scma. The other steps already have directories prepared.
cd /home/ckane/DAS/$1/cromwell_input/$2/scma/
mkdir normal/ tumor/
mv *NA* normal/
mv *TA* tumor/

# The Pipeline

## A. Sequence format conversion + marking adapters (SCMA) <a name='SCMA' />

Go to <a href=#tob>Table of Contents</a>

**Working directory**: /home/cromwell-scripts/seqconv-markadapt

**WDL**: seqConvMarkAdapt.wdl

**Input(s):**

example_scma_input.json
    note: update value of send_email and email to receive email on workflow completion
    
**Output(s):**

unaligned BAM (uBAM) with adapter sequence marked

fofn.list: a text file containing the path(s) to the generated uBAM(s).
used in lieu of an input.json in the next step (Pre-Processing).
Using a “file list” simplifies storage management.

1. Ensure you've prepared your input JSONS using the .csv and Python script. 

### Normal SCMA Submission

In [30]:
%%bash -s "$scma_normal_input_dir" "$group_name"

source activate
conda activate DNAseq

# change to SCMA working directory
cd /home/cromwell-scripts/seqconv-markadapt

# 2. Submit group of inputs to Cromwell with:
# ./group-submit-scma.sh -i /path/to/dir/with/input -g ${groupname}
./group-submit-scma.sh -i $1 -g $2_scma_normal

activate does not accept more than one argument:
['/home/ckane/DAS/dcr_1/cromwell_input/DCR-103_2/scma/normal', 'DCR-103_2']



╒══════════════════════════════════════╤═══════════╕
│ id                                   │ status    │
╞══════════════════════════════════════╪═══════════╡
│ f70121e6-2d53-4466-9ff1-c73f3faa1f87 │ Submitted │
╘══════════════════════════════════════╧═══════════╛
╒══════════════════════════════════════╤═══════════╕
│ id                                   │ status    │
╞══════════════════════════════════════╪═══════════╡
│ ddb68017-574b-4afb-84b3-1ee32337a674 │ Submitted │
╘══════════════════════════════════════╧═══════════╛
╒══════════════════════════════════════╤═══════════╕
│ id                                   │ status    │
╞══════════════════════════════════════╪═══════════╡
│ 7cd0895f-0374-40ea-8fcf-41aa07d92004 │ Submitted │
╘══════════════════════════════════════╧═══════════╛
╒══════════════════════════════════════╤═══════════╕
│ id                                   │ status    │
╞══════════════════════════════════════╪═══════════╡
│ 951b52e5-c054-4df0-8037-563d8420708e │ Submi

╒══════════════════════════════════════╤═══════════╕
│ id                                   │ status    │
╞══════════════════════════════════════╪═══════════╡
│ 007f4bfd-7214-421f-9657-4ca04570360e │ Submitted │
╘══════════════════════════════════════╧═══════════╛


In [3]:
%%bash -s "$group_name"

source activate
conda activate DNAseq

# check the progress of the submission using Oliver
# oliver st -g ${groupname}

oliver st -g $1_scma_normal

Could not find conda environment: DCR-103_2
You can list all discoverable environments with `conda info --envs`.



╒═══════════════════════╤═════════════╕
│ Job Group             │   Succeeded │
╞═══════════════════════╪═════════════╡
│ DCR-103_2_scma_normal │          32 │
╘═══════════════════════╧═════════════╛


In [3]:
%%bash

source activate
conda activate DNAseq

# paste in your desired workflow ID.
oliver inspect 

Group Name: DCR-090-102-105_scma_normal
Workflow Name: seqConvMarkAdapt
Workflow ID: ce957da0-27e0-43d1-9a35-9c1200454e9b
Workflow Version: WDL 1.0
Submission: Thu, Oct 19, 2023 12:39 AM
Start: Thu, Oct 19, 2023 12:39 AM
Duration: 7 minutes 26 seconds

╒════════════════════════════════════════════╤═══════════╤══════════╤════════════════════════════╤═════════════════════╕
│ Call Name                                  │   Attempt │ Status   │ Start                      │ Duration            │
╞════════════════════════════════════════════╪═══════════╪══════════╪════════════════════════════╪═════════════════════╡
│ seqConvMarkAdapt.PairedFastQsToUnmappedBAM │         1 │ Done     │ Thu, Oct 19, 2023 12:46 AM │ 4 minutes 1 seconds │
├────────────────────────────────────────────┼───────────┼──────────┼────────────────────────────┼─────────────────────┤
│ seqConvMarkAdapt.MarkIlluminaAdapters      │         1 │ Done     │ Thu, Oct 19, 2023 12:46 AM │ 3 minutes 3 seconds │
├────────────────────

### Tumor SCMA Submission

In [41]:
%%bash -s "$scma_tumor_input_dir" "$group_name"

source activate
conda activate DNAseq

cd /home/cromwell-scripts/seqconv-markadapt

# 2. Submit group of inputs to Cromwell with:

./group-submit-scma.sh -i $1 -g $2_scma_tumor

activate does not accept more than one argument:
['/home/ckane/DAS/dcr_1/cromwell_input/DCR-103_2/scma/tumor', 'DCR-103_2']



╒══════════════════════════════════════╤═══════════╕
│ id                                   │ status    │
╞══════════════════════════════════════╪═══════════╡
│ 0dd83971-72ef-4c8f-94a9-126f6bfa6a2d │ Submitted │
╘══════════════════════════════════════╧═══════════╛
╒══════════════════════════════════════╤═══════════╕
│ id                                   │ status    │
╞══════════════════════════════════════╪═══════════╡
│ 97e8f407-f504-4ab0-a129-5ec5bfcc70dc │ Submitted │
╘══════════════════════════════════════╧═══════════╛
╒══════════════════════════════════════╤═══════════╕
│ id                                   │ status    │
╞══════════════════════════════════════╪═══════════╡
│ f1cce044-a00e-4022-b063-b5e37b7ec8ae │ Submitted │
╘══════════════════════════════════════╧═══════════╛
╒══════════════════════════════════════╤═══════════╕
│ id                                   │ status    │
╞══════════════════════════════════════╪═══════════╡
│ 79080aeb-9822-48d1-af1a-9e0b467d4a17 │ Submi

╒══════════════════════════════════════╤═══════════╕
│ id                                   │ status    │
╞══════════════════════════════════════╪═══════════╡
│ a8949a2c-9a8b-42dd-ac71-17f7b4b28f6e │ Submitted │
╘══════════════════════════════════════╧═══════════╛


In [4]:
%%bash -s "$group_name"

source activate
conda activate DNAseq

# check the progress of the submission using Oliver
oliver st -g $1_scma_tumor

Could not find conda environment: DCR-103_2
You can list all discoverable environments with `conda info --envs`.



╒══════════════════════╤═════════════╕
│ Job Group            │   Succeeded │
╞══════════════════════╪═════════════╡
│ DCR-103_2_scma_tumor │          32 │
╘══════════════════════╧═════════════╛


In [ ]:
%%bash

source activate
conda activate DNAseq

# paste in your desired workflow ID.
oliver inspect 

it is recommended to submit tumor and normal fastqs as **separate groups (-N/T)** to simplify pointing to the tumor/normal files when generating input for PoN and M2.

It will be important for us to remember the groupname that we use when we utilize Oliver.

### Aggregate fofn.lists

### Normal

In [49]:
%%bash -s "$group_name" "$scma_normal_output_dir"

source activate
conda activate DNAseq

cd /home/cromwell-scripts/seqconv-markadapt

# Aggregate NORMALS fofn.list with:
# ./group-agg-scma_lists.sh -g ${groupname} -o ${output_directory}

./group-agg-scma_lists.sh -g $1_scma_normal -o $2

activate does not accept more than one argument:
['DCR-103_2', '/home/ckane/DAS/dcr_1/ubam/normal/DCR-103_2']



### Tumor

In [54]:
%%bash -s "$group_name" "$scma_tumor_output_dir"

source activate
conda activate DNAseq

cd /home/cromwell-scripts/seqconv-markadapt

# Aggregate TUMORS fofn.list with:
./group-agg-scma_lists.sh -g $1_scma_tumor -o $2

activate does not accept more than one argument:
['DCR-103_2', '/home/ckane/DAS/dcr_1/ubam/tumor/DCR-103_2']



## B. Pre-processing (PP) <a name='pre-processing' />

Go to <a href=#tob>Table of Contents</a>

Working directory: /home/cromwell-scripts/preprocessing

WDL: processing-for-variant-discovery.wdl

Input(s):

fofn.lists from SCMA.
Output(s):

aligned BAM file (b37/hg38)

### Normal PP Submission

In [4]:
%%bash -s "$pp_normal_input_dir" "$group_name"

source activate
conda activate DNAseq

# change to preprocessing working directory
cd /home/cromwell-scripts/preprocessing

# 1. Submit groups of uBAMs for pre-processing with:
# ./group-submit-pp.sh -i /path/to/dir/containing/scma/lists -g ${groupname} -r ${b37/hg38}

./group-submit-pp.sh -i $1 -g $2_pp_normal -r b37

activate does not accept more than one argument:
['/home/ckane/DAS/dcr_1/ubam/normal/DCR-103_2', 'DCR-103_2']



Submitting with default json /home/cromwell-scripts/preprocessing/default-preprocessing.b37.json
| id                                   | status    |
|:-------------------------------------|:----------|
| 2c70858c-97ad-49ed-8af1-9906d6a41c03 | Submitted |


In [3]:
%%bash -s "$group_name"

source activate
conda activate DNAseq

# check the progress of the submission using Oliver
oliver st -g $1_pp_normal

Could not find conda environment: DCR-103_2
You can list all discoverable environments with `conda info --envs`.



╒═════════════════════╤═══════════╤═════════════╕
│ Job Group           │   Aborted │   Succeeded │
╞═════════════════════╪═══════════╪═════════════╡
│ DCR-103_2_pp_normal │         1 │           1 │
╘═════════════════════╧═══════════╧═════════════╛


In [6]:
%%bash

source activate
conda activate DNAseq

# paste in your desired workflow ID.
oliver inspect c1e3835b-c984-4976-b0a6-244c9daf1e75

Group Name: DCR-002-0045-0056_pp_normal
Workflow Name: PreProcessingForVariantDiscovery_GATK4
Workflow ID: c1e3835b-c984-4976-b0a6-244c9daf1e75
Workflow Version: WDL 1.0
Submission: Wed, Nov 22, 2023 8:11 PM
Start: Wed, Nov 22, 2023 8:12 PM
Duration: 17 hours 38 minutes 24 seconds

╒══════════════════════════════════════════════════════════════════╤═══════════╤═════════╤══════════╤═══════════════════════════╤═══════════════════════════════╕
│ Call Name                                                        │   Attempt │   Shard │ Status   │ Start                     │ Duration                      │
╞══════════════════════════════════════════════════════════════════╪═══════════╪═════════╪══════════╪═══════════════════════════╪═══════════════════════════════╡
│ PreProcessingForVariantDiscovery_GATK4.CreateSequenceGroupingTSV │         1 │      -1 │ Done     │ Thu, Nov 23, 2023 1:16 PM │ 7 minutes 16 seconds          │
├──────────────────────────────────────────────────────────────────┼─

│ PreProcessingForVariantDiscovery_GATK4.SamToFastqAndBwaMem       │         1 │      14 │ Done     │ Thu, Nov 23, 2023 1:16 PM │ 2 hours 9 minutes 58 seconds  │
├──────────────────────────────────────────────────────────────────┼───────────┼─────────┼──────────┼───────────────────────────┼───────────────────────────────┤
│ PreProcessingForVariantDiscovery_GATK4.SamToFastqAndBwaMem       │         1 │      22 │ Done     │ Thu, Nov 23, 2023 1:16 PM │ 2 hours 11 minutes 42 seconds │
├──────────────────────────────────────────────────────────────────┼───────────┼─────────┼──────────┼───────────────────────────┼───────────────────────────────┤
│ PreProcessingForVariantDiscovery_GATK4.SamToFastqAndBwaMem       │         1 │      23 │ Done     │ Thu, Nov 23, 2023 1:16 PM │ 1 hours 52 minutes 44 seconds │
├──────────────────────────────────────────────────────────────────┼───────────┼─────────┼──────────┼───────────────────────────┼───────────────────────────────┤
│ PreProcessingForVariantDis

├──────────────────────────────────────────────────────────────────┼───────────┼─────────┼──────────┼───────────────────────────┼───────────────────────────────┤
│ PreProcessingForVariantDiscovery_GATK4.MergeBamAlignment         │         1 │       5 │ Done     │ Thu, Nov 23, 2023 1:16 PM │ 14 minutes 31 seconds         │
├──────────────────────────────────────────────────────────────────┼───────────┼─────────┼──────────┼───────────────────────────┼───────────────────────────────┤
│ PreProcessingForVariantDiscovery_GATK4.MarkDuplicates            │         1 │      -1 │ Done     │ Thu, Nov 23, 2023 1:16 PM │ 8 hours 32 minutes 37 seconds │
├──────────────────────────────────────────────────────────────────┼───────────┼─────────┼──────────┼───────────────────────────┼───────────────────────────────┤
│ PreProcessingForVariantDiscovery_GATK4.SortAndFixTags            │         1 │      -1 │ Done     │ Thu, Nov 23, 2023 1:16 PM │ 5 hours 10 minutes 59 seconds │
├───────────────────────────

│ PreProcessingForVariantDiscovery_GATK4.ApplyBQSR                 │         1 │       6 │ Done     │ Thu, Nov 23, 2023 1:16 PM │ 15 minutes 12 seconds         │
├──────────────────────────────────────────────────────────────────┼───────────┼─────────┼──────────┼───────────────────────────┼───────────────────────────────┤
│ PreProcessingForVariantDiscovery_GATK4.ApplyBQSR                 │         1 │       7 │ Done     │ Thu, Nov 23, 2023 1:16 PM │ 13 minutes 47 seconds         │
├──────────────────────────────────────────────────────────────────┼───────────┼─────────┼──────────┼───────────────────────────┼───────────────────────────────┤
│ PreProcessingForVariantDiscovery_GATK4.ApplyBQSR                 │         1 │      11 │ Done     │ Thu, Nov 23, 2023 1:16 PM │ 19 minutes 24 seconds         │
├──────────────────────────────────────────────────────────────────┼───────────┼─────────┼──────────┼───────────────────────────┼───────────────────────────────┤
│ PreProcessingForVariantDis

### Tumor PP Submission

In [56]:
%%bash -s "$pp_tumor_input_dir" "$group_name"

source activate
conda activate DNAseq

# change to preprocessing working directory
cd /home/cromwell-scripts/preprocessing

# 1. Submit groups of uBAMs for pre-processing with:
# pretty sure don't need nohup…&
./group-submit-pp.sh -i $1 -g $2_pp_tumor -r b37

activate does not accept more than one argument:
['/home/ckane/DAS/dcr_1/ubam/tumor/DCR-103_2', 'DCR-103_2']



Submitting with default json /home/cromwell-scripts/preprocessing/default-preprocessing.b37.json
| id                                   | status    |
|:-------------------------------------|:----------|
| 263e8e48-cbbc-47eb-b36b-46b8271f427a | Submitted |


In [58]:
%%bash -s "$group_name"

source activate
conda activate DNAseq

# check the progress of the submission using Oliver
oliver st -g $1_pp_tumor

Could not find conda environment: DCR-103_2
You can list all discoverable environments with `conda info --envs`.



╒════════════════════╤═══════════╕
│ Job Group          │   Running │
╞════════════════════╪═══════════╡
│ DCR-103_2_pp_tumor │         1 │
╘════════════════════╧═══════════╛


In [4]:
%%bash

source activate
conda activate DNAseq

# paste in your desired workflow ID.
oliver inspect d0b981c0-0c20-4c50-984d-25813c6eea3a

Group Name: DCR-0003-0011-0027_pp_tumor
Workflow Name: PreProcessingForVariantDiscovery_GATK4
Workflow ID: d0b981c0-0c20-4c50-984d-25813c6eea3a
Workflow Version: WDL 1.0
Submission: Thu, Nov 9, 2023 12:32 AM
Start: Thu, Nov 9, 2023 12:32 AM
Duration: 15 hours 31 minutes 28 seconds

╒══════════════════════════════════════════════════════════════════╤═══════════╤═════════╤══════════╤══════════════════════════╤═══════════════════════════════╕
│ Call Name                                                        │   Attempt │   Shard │ Status   │ Start                    │ Duration                      │
╞══════════════════════════════════════════════════════════════════╪═══════════╪═════════╪══════════╪══════════════════════════╪═══════════════════════════════╡
│ PreProcessingForVariantDiscovery_GATK4.CreateSequenceGroupingTSV │         1 │      -1 │ Done     │ Thu, Nov 9, 2023 3:24 PM │ 2 minutes 29 seconds          │
├──────────────────────────────────────────────────────────────────┼─────

├──────────────────────────────────────────────────────────────────┼───────────┼─────────┼──────────┼──────────────────────────┼───────────────────────────────┤
│ PreProcessingForVariantDiscovery_GATK4.SamToFastqAndBwaMem       │         1 │       7 │ Done     │ Thu, Nov 9, 2023 3:24 PM │ 3 hours 30 minutes 49 seconds │
├──────────────────────────────────────────────────────────────────┼───────────┼─────────┼──────────┼──────────────────────────┼───────────────────────────────┤
│ PreProcessingForVariantDiscovery_GATK4.SamToFastqAndBwaMem       │         1 │      10 │ Done     │ Thu, Nov 9, 2023 3:24 PM │ 2 hours 57 minutes 36 seconds │
├──────────────────────────────────────────────────────────────────┼───────────┼─────────┼──────────┼──────────────────────────┼───────────────────────────────┤
│ PreProcessingForVariantDiscovery_GATK4.MergeBamAlignment         │         1 │       0 │ Done     │ Thu, Nov 9, 2023 3:24 PM │ 21 minutes 3 seconds          │
├─────────────────────────────────

│ PreProcessingForVariantDiscovery_GATK4.MergeBamAlignment         │         1 │       9 │ Done     │ Thu, Nov 9, 2023 3:24 PM │ 9 minutes 16 seconds          │
├──────────────────────────────────────────────────────────────────┼───────────┼─────────┼──────────┼──────────────────────────┼───────────────────────────────┤
│ PreProcessingForVariantDiscovery_GATK4.MarkDuplicates            │         1 │      -1 │ Done     │ Thu, Nov 9, 2023 3:24 PM │ 6 hours 4 minutes 3 seconds   │
├──────────────────────────────────────────────────────────────────┼───────────┼─────────┼──────────┼──────────────────────────┼───────────────────────────────┤
│ PreProcessingForVariantDiscovery_GATK4.SortAndFixTags            │         1 │      -1 │ Done     │ Thu, Nov 9, 2023 3:24 PM │ 4 hours 10 minutes 18 seconds │
├──────────────────────────────────────────────────────────────────┼───────────┼─────────┼──────────┼──────────────────────────┼───────────────────────────────┤
│ PreProcessingForVariantDiscovery

├──────────────────────────────────────────────────────────────────┼───────────┼─────────┼──────────┼──────────────────────────┼───────────────────────────────┤
│ PreProcessingForVariantDiscovery_GATK4.ApplyBQSR                 │         1 │       4 │ Done     │ Thu, Nov 9, 2023 3:24 PM │ 10 minutes 26 seconds         │
├──────────────────────────────────────────────────────────────────┼───────────┼─────────┼──────────┼──────────────────────────┼───────────────────────────────┤
│ PreProcessingForVariantDiscovery_GATK4.ApplyBQSR                 │         1 │       5 │ Done     │ Thu, Nov 9, 2023 3:24 PM │ 10 minutes 15 seconds         │
├──────────────────────────────────────────────────────────────────┼───────────┼─────────┼──────────┼──────────────────────────┼───────────────────────────────┤
│ PreProcessingForVariantDiscovery_GATK4.ApplyBQSR                 │         1 │       6 │ Done     │ Thu, Nov 9, 2023 3:24 PM │ 10 minutes 47 seconds         │
├─────────────────────────────────

note: the -r flag selects reference files located in /home/isam/refs and specific default.json files. If a different reference is required, please create a modified copy of the group-submit-pp.sh script that points to an alternate default-processing.json and reference files.

Like in SCMA, it is recommended to submit tumor and normal fastqs as **separate groups (-N/T)** to simplify pointing to the tumor/normal files when generating input for PoN and M2.

### Aggregate PP results 

### Normal

In [4]:
%%bash -s "$group_name" "$pp_normal_output_dir"

source activate
conda activate DNAseq

cd /home/cromwell-scripts/

# 2. Aggregate NORMAL results with script from base cromwell-scripts directory:
# ./agg-grp-output.sh -g ${groupname} -o ${output_directory}

./agg-grp-output.sh -g $1_pp_normal -o $2

# NOTE: the output directory should be the one listed in the pon input json, where it will try to access the bam files.

activate does not accept more than one argument:
['DCR-103_2', '/home/ckane/DAS/dcr_1/bam/b37-n/DCR-103_2']

[I 240201 16:13:21 workflows:156] Found 1 eligible workflows given search criteria.


### Tumor

In [4]:
%%bash -s "$group_name" "$pp_tumor_output_dir"

source activate
conda activate DNAseq

cd /home/cromwell-scripts/

# 2. Aggregate TUMOR results with script from base cromwell-scripts directory:
./agg-grp-output.sh -g $1_pp_tumor -o $2

# NOTE: the output directory should be the one listed in the pon input json, where it will try to access the bam files.

activate does not accept more than one argument:
['DCR-002-0045-0056', '/home/ckane/DAS/crc_sub4/bam/b37-t/DCR-002-0045-0056']

[I 231125 22:00:11 workflows:156] Found 1 eligible workflows given search criteria.
[I 231125 22:00:11 workflows:156] Found 1 eligible workflows given search criteria.
[I 231125 22:00:11 workflows:156] Found 1 eligible workflows given search criteria.


## C. Generating Panel of Normals (PoN) <a name='pon' />

Go to <a href=#tob>Table of Contents</a>

Working directory: /home/cromwell-scripts/m2

WDL: mutect_pon.wdl

Input(s):

example_pon_input.json

note: update value of send_email and email to receive email on workflow completion

Output(s):

PoN

1. Ensure you've generated input.json. Make sure paths to reference files are correct. If WXS, identify Library Prep/Capture kit used. Set appropriate scatter_count.

Note that we should already have the input json ready to go, since the Python script at the beginning should have generated all the input files with the proper paths.

### Important notes

- If doing whole genome sequencing, be aware of the interval file that is used. This is specified in the config.ini file. If left blank, the Python script generates the .json with the path for the WES interval file.
- You should check the pon.json input file to confirm this.

In [5]:
%%bash -s "$pon_input_file" "$group_name"

source activate
conda activate DNAseq

# change to PoN working directory
cd /home/cromwell-scripts/m2

# 2. Run with:
# ./submit-pon.sh -i ${/path/to/input.json} -g ${groupname}

# ./submit-pon.sh -i $1 -g $2_pon
./submit-pon.sh -i /home/ckane/DAS/dcr_1/cromwell_input/DCR/pon.json -g DCR_6_pon

activate does not accept more than one argument:
['$pon_input_file', '$group_name']



╒══════════════════════════════════════╤═══════════╕
│ id                                   │ status    │
╞══════════════════════════════════════╪═══════════╡
│ 1dfa5642-0968-4105-b249-8018165ac884 │ Submitted │
╘══════════════════════════════════════╧═══════════╛


In [1]:
%%bash -s "$group_name"

source activate
conda activate DNAseq

# check the progress of the submission using Oliver
# oliver st -g $1_pon
oliver st -g DCR_6_pon

Could not find conda environment: $group_name
You can list all discoverable environments with `conda info --envs`.



╒═════════════╤═════════════╕
│ Job Group   │   Succeeded │
╞═════════════╪═════════════╡
│ DCR_6_pon   │           1 │
╘═════════════╧═════════════╛


In [2]:
%%bash

source activate
conda activate DNAseq

# paste in your desired workflow ID.
oliver inspect 1dfa5642-0968-4105-b249-8018165ac884

Group Name: DCR_6_pon
Workflow Name: Mutect2_Panel
Workflow ID: 1dfa5642-0968-4105-b249-8018165ac884
Workflow Version: WDL 1.0
Submission: Thu, Feb 22, 2024 2:39 PM
Start: Thu, Feb 22, 2024 2:39 PM
Duration: 1 days 21 hours 26 minutes 5 seconds

╒══════════════════════════════╤═══════════╤═════════╤══════════╤═══════════════════════════╤═══════════════════════════════════════╕
│ Call Name                    │   Attempt │   Shard │ Status   │ Start                     │ Duration                              │
╞══════════════════════════════╪═══════════╪═════════╪══════════╪═══════════════════════════╪═══════════════════════════════════════╡
│ Mutect2_Panel.Mutect2        │         1 │       0 │ Done     │ Thu, Feb 22, 2024 2:39 PM │ 7 hours 53 minutes 12 seconds         │
├──────────────────────────────┼───────────┼─────────┼──────────┼───────────────────────────┼───────────────────────────────────────┤
│ Mutect2_Panel.Mutect2        │         1 │       1 │ Done     │ Thu, Feb 22, 2024 

├──────────────────────────────┼───────────┼─────────┼──────────┼───────────────────────────┼───────────────────────────────────────┤
│ Mutect2_Panel.CreatePanel    │         1 │       2 │ Done     │ Thu, Feb 22, 2024 2:39 PM │ 15 minutes 11 seconds                 │
├──────────────────────────────┼───────────┼─────────┼──────────┼───────────────────────────┼───────────────────────────────────────┤
│ Mutect2_Panel.CreatePanel    │         1 │       3 │ Done     │ Thu, Feb 22, 2024 2:39 PM │ 15 minutes 3 seconds                  │
├──────────────────────────────┼───────────┼─────────┼──────────┼───────────────────────────┼───────────────────────────────────────┤
│ Mutect2_Panel.CreatePanel    │         1 │       7 │ Done     │ Thu, Feb 22, 2024 2:39 PM │ 12 minutes 15 seconds                 │
├──────────────────────────────┼───────────┼─────────┼──────────┼───────────────────────────┼───────────────────────────────────────┤
│ Mutect2_Panel.CreatePanel    │         1 │       9 │ Done   

In [3]:
%%bash

source activate
conda activate DNAseq

oliver logs a38a2ce7-87a3-4d76-9d8f-0c12b0de6bc6

╒══════════════════════════════╤═══════════╤═════════╤════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│ Call Name                    │   Attempt │   Shard │ Log Name   │ Location                                                                                                                                                                            │
╞══════════════════════════════╪═══════════╪═════════╪════════════╪═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│ Mutect2_Panel.PonComplete    │         1 │      -1 │ stdout     │ /home/cromwell-executions/Mutect2_Panel/a38a2ce7-87a3-4d76-9d8f-0c12b0de6bc6/call-PonComplete/execution/stdout                                                                      │


│ Mutect2.M2                   │         1 │       3 │ stderr     │ /home/cromwell-executions/Mutect2_Panel/a38a2ce7-87a3-4d76-9d8f-0c12b0de6bc6/call-Mutect2/shard-0/Mutect2/22dec982-6ed4-4498-a04e-236114165e13/call-M2/shard-3/execution/stderr     │
├──────────────────────────────┼───────────┼─────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Mutect2.M2                   │         1 │       4 │ stdout     │ /home/cromwell-executions/Mutect2_Panel/a38a2ce7-87a3-4d76-9d8f-0c12b0de6bc6/call-Mutect2/shard-0/Mutect2/22dec982-6ed4-4498-a04e-236114165e13/call-M2/shard-4/execution/stdout     │
├──────────────────────────────┼───────────┼─────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤


├──────────────────────────────┼───────────┼─────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Mutect2.M2                   │         1 │       3 │ stdout     │ /home/cromwell-executions/Mutect2_Panel/a38a2ce7-87a3-4d76-9d8f-0c12b0de6bc6/call-Mutect2/shard-1/Mutect2/6e976914-e7df-425a-a984-438c1eb24a85/call-M2/shard-3/execution/stdout     │
├──────────────────────────────┼───────────┼─────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Mutect2.M2                   │         1 │       3 │ stderr     │ /home/cromwell-executions/Mutect2_Panel/a38a2ce7-87a3-4d76-9d8f-0c12b0de6bc6/call-Mutect2/shard-1/Mutect2/6e976914-e7df-425a-a984-438c1eb24a85/call-M2/shard-3/execution/stderr     │


│ Mutect2.M2                   │         1 │       2 │ stdout     │ /home/cromwell-executions/Mutect2_Panel/a38a2ce7-87a3-4d76-9d8f-0c12b0de6bc6/call-Mutect2/shard-2/Mutect2/0319f9ad-2be6-4919-ae2f-12a4214bd304/call-M2/shard-2/execution/stdout     │
├──────────────────────────────┼───────────┼─────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Mutect2.M2                   │         1 │       2 │ stderr     │ /home/cromwell-executions/Mutect2_Panel/a38a2ce7-87a3-4d76-9d8f-0c12b0de6bc6/call-Mutect2/shard-2/Mutect2/0319f9ad-2be6-4919-ae2f-12a4214bd304/call-M2/shard-2/execution/stderr     │
├──────────────────────────────┼───────────┼─────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤


├──────────────────────────────┼───────────┼─────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Mutect2_Panel.CreatePanel    │         1 │       4 │ stderr     │ /home/cromwell-executions/Mutect2_Panel/a38a2ce7-87a3-4d76-9d8f-0c12b0de6bc6/call-CreatePanel/shard-4/execution/stderr                                                              │
├──────────────────────────────┼───────────┼─────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Mutect2_Panel.CreatePanel    │         1 │       5 │ stdout     │ /home/cromwell-executions/Mutect2_Panel/a38a2ce7-87a3-4d76-9d8f-0c12b0de6bc6/call-CreatePanel/shard-5/execution/stdout                                                              │


│ Mutect2_Panel.CreatePanel    │         1 │      12 │ stderr     │ /home/cromwell-executions/Mutect2_Panel/a38a2ce7-87a3-4d76-9d8f-0c12b0de6bc6/call-CreatePanel/shard-12/execution/stderr                                                             │
├──────────────────────────────┼───────────┼─────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Mutect2_Panel.CreatePanel    │         1 │      13 │ stdout     │ /home/cromwell-executions/Mutect2_Panel/a38a2ce7-87a3-4d76-9d8f-0c12b0de6bc6/call-CreatePanel/shard-13/execution/stdout                                                             │
├──────────────────────────────┼───────────┼─────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤


├──────────────────────────────┼───────────┼─────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Mutect2_Panel.CreatePanel    │         1 │      21 │ stdout     │ /home/cromwell-executions/Mutect2_Panel/a38a2ce7-87a3-4d76-9d8f-0c12b0de6bc6/call-CreatePanel/shard-21/execution/stdout                                                             │
├──────────────────────────────┼───────────┼─────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Mutect2_Panel.CreatePanel    │         1 │      21 │ stderr     │ /home/cromwell-executions/Mutect2_Panel/a38a2ce7-87a3-4d76-9d8f-0c12b0de6bc6/call-CreatePanel/shard-21/execution/stderr                                                             │


### Aggregate pon file

In [3]:
%%bash -s "$group_name" "$pon_output_dir"

source activate
conda activate DNAseq

# change to PoN working directory
cd /home/cromwell-scripts/m2

# 3. Aggregate pon file with:
# ./agg-pon.sh -g ${groupname} -o ${/dir/to/output/pon}

# ./agg-pon.sh -g $1_pon -o $2
./agg-pon.sh -g DCR_6_pon -o /home/ckane/DAS/dcr_1/pon/

activate does not accept more than one argument:
['$group_name', '$pon_output_dir']



# D. Variant calling with Mutect2 (M2) <a name='mutect' />

Go to <a href=#tob>Table of Contents</a>

Working directory: /home/cromwell-scripts/m2

WDL: mutect2.wdl

Input(s):

example_m2_input.json

note: update value of send_email and email to receive email on workflow completion

Output(s):

Sample mutation calls in MAF format

1. Ensure you generated the proper input.json using notebook. Some notes to consider…

Add location of PoN file generated from section C

Make sure paths to reference files are correct

If WXS, identify Library Prep/Capture kit used (link to notes on obtaining kit).

Note: Mutation calling can be made stricter or more permissive by modifying Mutect2.m2_extra_args parameter in the notebook.

### Important notes

- If doing whole genome sequencing, be aware of the interval file that is used. This is specified in the config.ini file. If left blank, the Python script generates the .json with the path for the WES interval file.
- You should check the m2.json input file to confirm this.

In [8]:
%%bash -s "$m2_input_dir" "$group_name"

source activate
conda activate DNAseq

# change to mutect working directory
cd /home/cromwell-scripts/m2

# 2. Run variant calling with:
# ./group-submit-m2.sh -i ${/full/path/to/inputs/dir} -g ${groupname}

# ./group-submit-m2.sh -i $1 -g $2_mutect_2
# ./group-submit-m2.sh -i /home/ckane/crc_sub3/cromwell_input/DCR103/m2 -g $2_mutect_2

./group-submit-m2.sh -i /home/ckane/DAS/dcr_1/cromwell_input/DCR-090-102-105_2/m2 -g DCR-090-102-105_2_mutect_2

activate does not accept more than one argument:
['$m2_input_dir', '$group_name']



| id                                   | status    |
|:-------------------------------------|:----------|
| da367131-0f38-4a1f-9569-150e37ec3aa1 | Submitted |
| id                                   | status    |
|:-------------------------------------|:----------|
| 85ab3c51-befc-418d-bc6c-fe34f9dadf88 | Submitted |
| id                                   | status    |
|:-------------------------------------|:----------|
| 5d409eb6-2198-4a0f-8ef8-5701740ae2bc | Submitted |


In [1]:
%%bash -s "$group_name"

source activate
conda activate DNAseq

# check the progress of the submission using Oliver
# oliver st -g $1_mutect_2
oliver st -g DCR-090-102-105_2_mutect_2

Could not find conda environment: $group_name
You can list all discoverable environments with `conda info --envs`.



╒════════════════════════════╤═════════════╕
│ Job Group                  │   Succeeded │
╞════════════════════════════╪═════════════╡
│ DCR-090-102-105_2_mutect_2 │           3 │
╘════════════════════════════╧═════════════╛


In [18]:
%%bash

source activate
conda activate DNAseq

# paste in your desired workflow ID.
oliver inspect da367131-0f38-4a1f-9569-150e37ec3aa1

Group Name: DCR-090-102-105_2_mutect_2
Workflow Name: Mutect2
Workflow ID: da367131-0f38-4a1f-9569-150e37ec3aa1
Workflow Version: WDL 1.0
Submission: Wed, Feb 28, 2024 10:42 PM
Start: Wed, Feb 28, 2024 10:42 PM
Duration: 5 minutes 8 seconds (In progress)

╒════════════════════════╤═══════════╤═════════╤══════════╤════════════════════════════╤════════════════════════════════════╕
│ Call Name              │   Attempt │   Shard │ Status   │ Start                      │ Duration                           │
╞════════════════════════╪═══════════╪═════════╪══════════╪════════════════════════════╪════════════════════════════════════╡
│ Mutect2.SplitIntervals │         1 │      -1 │ Done     │ Wed, Feb 28, 2024 10:43 PM │ 24 seconds                         │
├────────────────────────┼───────────┼─────────┼──────────┼────────────────────────────┼────────────────────────────────────┤
│ Mutect2.M2             │         1 │       1 │ Running  │ Wed, Feb 28, 2024 10:43 PM │ 4 minutes 34 seconds (In 

│ Mutect2.M2             │         1 │      37 │ Running  │ Wed, Feb 28, 2024 10:43 PM │ 4 minutes 33 seconds (In progress) │
├────────────────────────┼───────────┼─────────┼──────────┼────────────────────────────┼────────────────────────────────────┤
│ Mutect2.M2             │         1 │      38 │ Running  │ Wed, Feb 28, 2024 10:43 PM │ 4 minutes 33 seconds (In progress) │
├────────────────────────┼───────────┼─────────┼──────────┼────────────────────────────┼────────────────────────────────────┤
│ Mutect2.M2             │         1 │      40 │ Running  │ Wed, Feb 28, 2024 10:43 PM │ 4 minutes 33 seconds (In progress) │
├────────────────────────┼───────────┼─────────┼──────────┼────────────────────────────┼────────────────────────────────────┤
│ Mutect2.M2             │         1 │       0 │ Running  │ Wed, Feb 28, 2024 10:43 PM │ 4 minutes 33 seconds (In progress) │
├────────────────────────┼───────────┼─────────┼──────────┼────────────────────────────┼──────────────────────────────

In [2]:
%%bash

source activate
conda activate DNAseq

oliver logs 380309a6-e241-4085-974c-09c76710b42f

╒════════════════════════╤═══════════╤═════════╤════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│ Call Name              │   Attempt │   Shard │ Log Name   │ Location                                                                                                    │
╞════════════════════════╪═══════════╪═════════╪════════════╪═════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│ Mutect2.SplitIntervals │         1 │      -1 │ stdout     │ /home/cromwell-executions/Mutect2/380309a6-e241-4085-974c-09c76710b42f/call-SplitIntervals/execution/stdout │
├────────────────────────┼───────────┼─────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Mutect2.SplitIntervals │         1 │      -1 │ stderr     │ /home/cromwell-executions/Mutect2/380309a6-e241-4085-974c-09c76710b42f/call-Sp

├────────────────────────┼───────────┼─────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Mutect2.M2             │         1 │      10 │ stderr     │ /home/cromwell-executions/Mutect2/380309a6-e241-4085-974c-09c76710b42f/call-M2/shard-10/execution/stderr    │
├────────────────────────┼───────────┼─────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Mutect2.M2             │         1 │      11 │ stdout     │ /home/cromwell-executions/Mutect2/380309a6-e241-4085-974c-09c76710b42f/call-M2/shard-11/execution/stdout    │
├────────────────────────┼───────────┼─────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Mutect2.M2             │         1 │      11 │ stderr     │ /home/cromwell-executions/Mutect2/380309a6-e241-4085-974c-09c76710b42f/call-M2

├────────────────────────┼───────────┼─────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Mutect2.M2             │         1 │      22 │ stderr     │ /home/cromwell-executions/Mutect2/380309a6-e241-4085-974c-09c76710b42f/call-M2/shard-22/execution/stderr    │
├────────────────────────┼───────────┼─────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Mutect2.M2             │         1 │      23 │ stdout     │ /home/cromwell-executions/Mutect2/380309a6-e241-4085-974c-09c76710b42f/call-M2/shard-23/execution/stdout    │
├────────────────────────┼───────────┼─────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Mutect2.M2             │         1 │      23 │ stderr     │ /home/cromwell-executions/Mutect2/380309a6-e241-4085-974c-09c76710b42f/call-M2

├────────────────────────┼───────────┼─────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Mutect2.M2             │         1 │      34 │ stderr     │ /home/cromwell-executions/Mutect2/380309a6-e241-4085-974c-09c76710b42f/call-M2/shard-34/execution/stderr    │
├────────────────────────┼───────────┼─────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Mutect2.M2             │         1 │      35 │ stdout     │ /home/cromwell-executions/Mutect2/380309a6-e241-4085-974c-09c76710b42f/call-M2/shard-35/execution/stdout    │
├────────────────────────┼───────────┼─────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Mutect2.M2             │         1 │      35 │ stderr     │ /home/cromwell-executions/Mutect2/380309a6-e241-4085-974c-09c76710b42f/call-M2

### Aggregate mutect outputs

In [2]:
%%bash -s "$group_name" "$m2_output_dir"

source activate
conda activate DNAseq

# change to mutect working directory
cd /home/cromwell-scripts/

# 3. Aggregate results with /home/cromwell-scripts/agg-grp-output.sh:
# ./agg-grp-output.sh -g ${groupname} -o ${/full/path/to/output/dir}

# ./agg-grp-output.sh -g $1_mutect_2 -o $2
# ./agg-grp-output.sh -g $1_mutect_2 -o /home/ckane/crc_sub3/maf
# ./agg-grp-output.sh -g DCR-0003-0011-0027_2_mutect -o /home/ckane/DAS/dcr_1/maf/DCR-0003-0011-0027_2
# ./agg-grp-output.sh -g DCR-002-0045-0056_2_mutect -o /home/ckane/DAS/dcr_1/maf/DCR-002-0045-0056_2
./agg-grp-output.sh -g DCR-090-102-105_2_mutect_2 -o /home/ckane/DAS/dcr_1/maf/DCR-090-102-105_2

activate does not accept more than one argument:
['$group_name', '$m2_output_dir']

[I 240229 15:48:32 workflows:156] Found 1 eligible workflows given search criteria.
[I 240229 15:48:32 workflows:156] Found 1 eligible workflows given search criteria.
[I 240229 15:48:32 workflows:156] Found 1 eligible workflows given search criteria.


# E. Aggregating MAFs <a name='agg-maf' />

Go to <a href=#tob>Table of Contents</a>

Working directory: /home/cromwell-scripts

Input(s):

collection of individual MAFs from variant calling
Output(s):

An .aggregated.maf file


1. Using editors like vi or less check the number of meta lines preceding the header in the MAF files generated from variant calling. Each MAF from the same project run should have the same number of meta lines. The number of meta lines in a typical Funcotated MAF is 159, (160 is where the header starts), however, results can vary depending on the source material.

In [ ]:
# count the number of meta lines in the .maf file
# meta lines are those that precede the header (with all the features and stuff)

# meta_count = !ls count…

# I found 160 meta lines in each file for thd DCR-090… run

In [3]:
%%bash

# change to cromwell-scripts directory
cd /home/cromwell-scripts

# 2. Using the number of meta lines in step 1, run:
# ./maf_concatenator.sh -m ${/path/to/mafs} -n ${number_of_meta_lines+1} -o ${/path/to/output/dir} -p ${project_name}
./maf_concatenator.sh -m /home/ckane/DAS/dcr_1/maf/DCR -n 161 -o /home/ckane/DAS/dcr_1/maf/DCR_agg -p DCR_wgs

# change this to use the variables next time.

# Capture Kit/Interval list (if WXS)

1. Common kits are Agilent SureSelect. Converted bed files for these are available in ref_dir. Files must be converted to use naming convention (e.g. chromosome 1 is 1 not chr1).

2. Link to agilent etc

SureSelect Human All Exon V5

Downloading from Agilent and as described here: 
“Find Designs” → “Agilent Catalog”

Most likely will be interested in *[design_ID]_Regions.bed*

BED file is at /home/imibrahim/refs/Agilent_SureSelect_Human_All_Exon_V5

In [ ]:
%%bash

# 3. To convert run:

$ sed -i 's/chr//g' S04380110_Regions.bed

Before loading to db, check how unique the subject names are. Naming conventions like “Patient 1” would necessitate some

